In [9]:
import pandas as pd
import numpy as np
import warnings
warnings. simplefilter(action='ignore', category=Warning)

In [10]:
def GoldenCrossverSignal(name):
    path = f'../2)data/{name}.csv'
    data = pd.read_csv(path, parse_dates=['Date'], index_col='Date')
    data['Prev_Close'] = data.Close.shift(1) #we use prev_close to remove forwad bias
    data['50_DMA'] = data.Prev_Close.rolling(window=50, min_periods=1).mean()
    data['200_DMA'] = data.Prev_Close.rolling(window=200, min_periods=1).mean()
    data['Signal'] = 0
    data['Signal'] = np.where(data['50_DMA'] > data['200_DMA'], 1, 0)
    data['Position'] = data.Signal.diff()
    df_pos = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
    return df_pos

In [11]:
data = GoldenCrossverSignal('TATAMOTORS')

In [12]:
req_data = data[(data.index >= data[data['Position'] == 'Buy'].index[0]) & (data.index <= data[data['Position'] == 'Sell'].index[-1])]

In [13]:
# Name, Entry TIme, Entry PRice, QTY, Exit Time, Exit Price
class Backtest:
    def __init__(self):
        self.columns = ['Equity Name', 'Trade', 'Entry Time', 'Entry Price', 'Exit Time', 'Exit Price', 'Quantity', 'Position Size', 'PNL', '% PNL']
        self.backtesting = pd.DataFrame(columns=self.columns)

    def buy(self, equity_name, entry_time, entry_price, qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Long Open'
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        self.trade_log['Equity Name'] = equity_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price, 2)

    def sell(self, exit_time, exit_price, exit_type, charge):
        self.trade_log['Trade'] = 'Long Closed'
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price, 2)
        self.trade_log['Exit Type'] = exit_type
        self.trade_log['PNL'] = round((self.trade_log['Exit Price'] - self.trade_log['Entry Price']) * self.trade_log['Quantity'] - charge, 3)
        self.trade_log['% PNL'] = round((self.trade_log['PNL'] / self.trade_log['Position Size']) * 100, 3)
        self.trade_log['Holding Period'] = exit_time - self.trade_log['Entry Time']
        new = pd.DataFrame([self.trade_log])
        self.backtesting = pd.concat([new,self.backtesting], ignore_index=True)

In [14]:
bt = Backtest()
capital = 10000
for index, data in req_data.iterrows():
    if(data.Position == 'Buy'):
        qty = capital // data.Open
        bt.buy('TATAMOTORS', index, data.Open, qty)
    else:
        bt.sell(index, data.Open, 'Exit Trigger', 0)

In [15]:
bt.backtesting

,Equity Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,PNL,% PNL,Exit Type,Holding Period
0,TATAMOTORS,Long Closed,2022-09-14,447.60,2022-10-11,397.00,22.0,9847.200,-1113.20,-11.305,Exit Trigger,27 days
1,TATAMOTORS,Long Closed,2020-09-21,147.75,2022-05-19,400.00,67.0,9899.250,16900.75,170.728,Exit Trigger,605 days
2,TATAMOTORS,Long Closed,2019-12-26,175.90,2020-03-25,67.75,56.0,9850.400,-6056.40,-61.484,Exit Trigger,90 days
3,TATAMOTORS,Long Closed,2017-02-01,527.50,2017-03-15,475.00,18.0,9495.000,-945.00,-9.953,Exit Trigger,42 days
4,TATAMOTORS,Long Closed,2016-05-04,405.50,2017-01-18,523.05,24.0,9732.000,2821.20,28.989,Exit Trigger,259 days
5,TATAMOTORS,Long Closed,2013-09-11,344.80,2015-06-05,445.65,29.0,9999.248,2924.65,29.249,Exit Trigger,632 days
6,TATAMOTORS,Long Closed,2012-10-23,263.18,2013-08-14,295.08,37.0,9737.555,1180.30,12.121,Exit Trigger,295 days
7,TATAMOTORS,Long Closed,2012-02-01,241.26,2012-08-03,218.65,41.0,9891.751,-927.01,-9.372,Exit Trigger,184 days
8,TATAMOTORS,Long Closed,2009-05-29,67.16,2011-06-14,199.56,148.0,9939.627,19595.20,197.142,Exit Trigger,746 days
9,TATAMOTORS,Long Closed,2007-11-06,141.84,2008-03-14,119.58,70.0,9928.477,-1558.20,-15.694,Exit Trigger,129 days


In [16]:
bt.backtesting.to_csv('TATAMOTORS_TEST.csv')
bt.backtesting.PNL.sum()

52684.65000000001